### xgboost model to predict mean player outcomes
used in conjunction with nn algs to determine mean wrc for player. Three diffrent models based on available data then combinded to create an xgb predection.

In [47]:
import pandas as pd
import os
import numpy as np

os.chdir('C:/Users/dalto/OneDrive/Pictures/Documents/Projects/MLB Season Model/')

In [48]:
# import data
data_02_24 = pd.read_csv('./data/cleaned/data_02_24.csv')
df = data_02_24.copy() # import data
df = df.drop(columns=['Unnamed: 0'])

#### find the z_score of each stat
weighted avg of year, std of year, 10 away from 100 is one std


In [49]:
def mean_year(group):
        avg = np.average(group, weights=group.index.map(lambda x: df.loc[x, 'PA']))
        return avg

In [50]:
# weighted std to account for low PA outlier preformances
def calculate_weighted_std(group):
    avg = np.average(group, weights=group.index.map(lambda x: df.loc[x, 'PA']))
    variance = np.average((group - avg)**2, weights=group.index.map(lambda x: df.loc[x, 'PA']))
    return np.sqrt(variance)

In [51]:
def z_scores(player_df, mean_df, std_df):
    # z scores for each stat and player, also creating a new df flow, add back names based on MLBAMID later
    z_scores = pd.DataFrame(index=player_df.index, columns=player_df.select_dtypes(include=[np.number]).columns)

    # scaled for each col
    for col in player_df.select_dtypes(include=[np.number]).columns:
        if col not in ['Season', 'MLBAMID']:  # not needed
            for idx in player_df.index:
                season = player_df.loc[idx, 'Season']
                value = player_df.loc[idx, col]
                mean = mean_df.loc[mean_df['Season'] == season, col].iloc[0]
                std = std_df.loc[std_df['Season'] == season, col].iloc[0]
                # scale so that 10 is 1 std away
                z_scores.loc[idx, col] = 100 + ((value - mean) / std * 10)

    z_scores['Season'] = player_df['Season']
    
    # cols to numeric
    for col in z_scores.columns:
        if col not in ['Season', 'MLBAMID']:
            z_scores[col] = pd.to_numeric(z_scores[col])
    
    # reinsert player names
    z_scores.insert(0, 'Name', player_df['Name']) 

    return z_scores

In [52]:
# df to store mean results
mean_values = pd.DataFrame(index=df['Season'].unique())

for col in df.select_dtypes(include=[np.number]).columns:
    if col != 'Season' and col != 'MLBAMID':  # skip meaningless numeric
        yearly_mean = df.groupby('Season')[col].apply(mean_year)
        mean_values[col] = yearly_mean

# reset and rename index
mean_values = mean_values.reset_index()
mean_values = mean_values.rename(columns={'index': 'Season'})

# sort and display
mean_values = mean_values.sort_values(by=['Season'])
mean_values.head()

,Season,PA,Age,BB%,K%,BABIP,wRC+,BsR,Off,Def,WAR,O-Swing%,Z-Swing%,O-Contact%,Z-Contact%,Contact%,SwStr%,CSW%
19,2002,473.684181,29.200233,0.088931,0.162881,0.294877,99.856949,0.221509,3.922156,-0.767819,1.888873,0.181982,0.703447,0.507012,0.860107,0.797336,0.094299,0.256508
22,2003,475.461939,29.191111,0.086575,0.159114,0.296402,99.927082,0.141683,4.016513,-0.608826,1.909976,0.221738,0.691527,0.527634,0.882055,0.800120,0.093138,0.260791
20,2004,477.651063,29.335336,0.087845,0.163399,0.299236,99.906874,0.255112,4.303784,-0.517574,1.945867,0.204546,0.693372,0.623672,0.855852,0.806547,0.088749,0.255794
14,2005,475.451649,29.259353,0.083354,0.159429,0.297611,100.116588,0.277356,3.656253,-0.893002,1.858494,0.214757,0.681891,0.550610,0.886206,0.813783,0.086017,0.255201
15,2006,483.988528,29.191618,0.085995,0.162801,0.303676,100.090682,0.168365,3.574442,-0.760455,1.862631,0.232215,0.671328,0.584224,0.889358,0.816041,0.084597,0.251676


In [53]:
# Create empty DataFrame to store results
std_values = pd.DataFrame(index=df['Season'].unique())

# Loop through numeric columns
for col in df.select_dtypes(include=[np.number]).columns:
    if col != 'Season' and col != 'MLBAMID':  # skip meaningless numeric
        yearly_std = df.groupby('Season')[col].apply(calculate_weighted_std)
        std_values[col] = yearly_std

std_values = std_values.reset_index()
std_values = std_values.rename(columns={'index': 'Season'})

std_values = std_values.sort_values(by=['Season'])
std_values.head()

,Season,PA,Age,BB%,K%,BABIP,wRC+,BsR,Off,Def,WAR,O-Swing%,Z-Swing%,O-Contact%,Z-Contact%,Contact%,SwStr%,CSW%
19,2002,184.243302,4.009692,0.037986,0.055977,0.035674,31.645660,3.121745,19.286650,9.129972,2.195603,0.047095,0.056035,0.108927,0.051809,0.061293,0.032167,0.028209
22,2003,185.106060,4.078477,0.035647,0.053696,0.034696,30.915634,3.282536,17.866265,9.428022,2.134635,0.048461,0.058644,0.107443,0.047224,0.060704,0.031931,0.027495
20,2004,189.820910,4.146485,0.037057,0.057978,0.036772,30.199233,3.575905,17.922925,9.712096,2.177108,0.051101,0.059314,0.106975,0.055553,0.063824,0.033339,0.030215
14,2005,182.884548,4.155987,0.032447,0.054885,0.035186,28.238546,3.328612,16.513000,10.060394,1.953790,0.057376,0.059317,0.117271,0.047543,0.061264,0.032130,0.027910
15,2006,184.138008,4.310582,0.035272,0.055416,0.036208,28.100105,3.600161,16.729148,9.906593,1.979241,0.060376,0.059520,0.112736,0.046223,0.061019,0.031551,0.028022


apply z scores

In [54]:
z_score_df = z_scores(player_df=df, mean_df=mean_values, std_df=std_values)
z_score_df.head()

,Name,Season,PA,Age,BB%,K%,BABIP,wRC+,BsR,Off,Def,WAR,O-Swing%,Z-Swing%,O-Contact%,Z-Contact%,Contact%,SwStr%,CSW%
0,brent clevlen,2007,74.826870,85.958971,74.686219,193.060161,107.344122,43.282571,98.653170,96.363387,100.657718,89.901996,138.703183,88.950287,71.666459,22.211544,37.187201,169.893575,204.596460
1,brett hayes,2009,74.297907,90.367791,72.475999,127.859623,108.219882,117.931506,99.866315,98.235778,100.843512,91.580480,170.668602,80.735141,60.421763,105.381033,65.578050,147.469352,199.428878
2,carlos peguero,2014,75.723755,96.257739,107.348483,147.612705,148.436984,92.738037,100.377615,97.711317,100.971051,91.256604,122.457976,73.452584,41.259125,91.336666,33.462464,156.578380,183.278087
3,seby zavala,2019,75.887131,92.057094,73.040518,181.646207,108.098997,44.146682,99.594613,95.874994,101.729855,90.245625,129.458257,111.272068,55.708080,34.476706,40.158323,179.134016,179.993300
4,franklin barreto,2020,77.834198,88.981943,76.647649,142.109822,78.737137,59.956124,101.610132,90.946521,99.887662,86.747334,120.942395,87.572409,55.233155,84.442687,56.517822,144.969583,165.941780


#### xgboost model

In [101]:
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import root_mean_squared_error

make wrc+ value the next year

In [103]:
# remove 2024 to train model bc we do not know the following year
train_z = z_score_df[z_score_df['Season'] < 2024].copy()
train_z['next_wrc+'] = np.nan # empty col


for idx in train_z.index:
    # retrive name, season
    name = train_z.loc[idx, 'Name']
    season = train_z.loc[idx, 'Season']
    # full df
    next_year_noscale = df[(df['Name'] == name) & (df['Season'] == season + 1)]
    next_year_scaled = z_score_df[(z_score_df['Name'] == name) & (z_score_df['Season'] == season + 1)]

    # add to df
    if not next_year_noscale.empty:
        train_z.loc[idx, 'next_wrc+'] = next_year_noscale['wRC+'].iloc[0]
        train_z.loc[idx, 'next_pa'] = next_year_scaled['PA'].iloc[0] # need for later anaylsis

# remove rows of nas (player retired or smth)
train_z = train_z.dropna(subset=['next_wrc+'])

# account for outliers
train_z = train_z.sort_values(by='next_wrc+')
train_z = train_z[500:-500] # try to account for outliers

# display
train_z.head()

,Name,Season,PA,Age,BB%,K%,BABIP,wRC+,BsR,Off,...,WAR,O-Swing%,Z-Swing%,O-Contact%,Z-Contact%,Contact%,SwStr%,CSW%,next_wrc+,next_pa
3416,chris stewart,2017,82.279879,117.502732,92.552812,90.839381,80.316000,72.962937,100.556947,88.625777,...,87.745555,84.458930,84.658307,103.654620,110.440367,111.556269,86.652033,108.740395,15.227124,75.534893
9912,tres barrera,2021,81.846332,93.287749,107.644998,96.865723,108.357987,102.322820,98.460116,97.905655,...,94.656235,97.068794,107.451723,93.853736,107.760439,103.554104,98.114792,92.767708,15.285413,78.635476
1146,dan johnson,2014,77.803052,114.131552,122.452953,101.557871,85.650532,97.826398,96.133996,96.635333,...,89.666201,93.314924,82.526230,98.332139,84.710469,89.919826,102.199509,118.985169,15.322849,75.949322
13659,juan graterol,2016,76.060754,96.262838,73.235348,99.075631,115.364903,96.718893,99.751250,97.256369,...,90.793509,124.067143,96.243400,124.364690,95.882963,111.767429,91.869366,64.126751,15.460515,79.080143
2039,danny ardoin,2005,87.563102,101.782120,99.165096,121.644570,102.545828,89.474349,93.746442,91.109483,...,95.681722,103.288295,97.656807,93.692002,75.733228,81.390528,115.814894,119.147674,15.816693,81.047447


dropped top and bottom 500 which may seem arb but it resulted in the wRC+ mean back at 100 and logically I like the look of the df a lot more. I am also making the choice of letter the model see the next year PA. I will build a seperate model to predict PA which I think is likely the most important step. without this it does not have context for injury. this is a important decison so I will experiemnt with it

In [104]:
X = train_z.drop(columns=['Name', 'Season', 'BsR', 'Off', 'Def', 'next_wrc+'])
Y = train_z['next_wrc+']
xtrain, xtest, ytrain, ytest = train_test_split(X, Y, test_size=0.25, random_state=26)

model

In [105]:
model = xgb.XGBRegressor(n_jobs=-1, n_estimators=50)
model.fit(xtrain, ytrain)
print(model.feature_importances_)

[0.03015881 0.03175204 0.04435361 0.03493639 0.04043832 0.07789238
 0.0446389  0.03785243 0.04332533 0.04140817 0.03745141 0.05006407
 0.03959433 0.04590441 0.40022948]


save results

In [106]:
results_df = xtest.copy()
results_df['Name'] = train_z.loc[xtest.index, 'Name']
results_df['Season'] = train_z.loc[xtest.index, 'Season']
results_df['next_pa'] = train_z.loc[xtest.index, 'next_pa']
results_df['pred_wrc+'] = model.predict(xtest)
results_df['season_plus1_wrc+'] = ytest
results_df = results_df[['Name', 'Season', 'pred_wrc+', 'season_plus1_wrc+', 'next_pa']]
results_df['miss'] = (results_df['season_plus1_wrc+'] - results_df['pred_wrc+']).abs()
results_df.head()

,Name,Season,pred_wrc+,season_plus1_wrc+,next_pa,miss
11453,javier valentin,2006,78.294754,80.802973,88.504314,2.508219
12433,shea hillenbrand,2004,106.342834,108.601368,109.270786,2.258534
10194,michael martinez,2014,53.010242,67.626986,76.556207,14.616744
4931,jonathan herrera,2012,62.947689,74.699519,86.836808,11.751830
5643,zack collins,2021,83.327911,56.414984,81.707790,26.912928


error

In [107]:
ypred = model.predict(xtest)
rmse = root_mean_squared_error(y_pred=ypred, y_true=ytest)
print(f"rmse for test set {rmse}")

rmse for test set 23.03266364646291
